In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env')

import os

In [2]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.docstore.document import Document

In [12]:
class BaseStructureChain:
    PROMPT = ""

    def __init__(self):
        self.llm = ChatOpenAI()
        self.chain = LLMChain.from_string(
            llm=self.llm,
            template=self.PROMPT,
        )

        self.chain.verbose = True

class BaseEventChain:
    PROMPT = ""

    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
        self.chain = LLMChain.from_string(
            llm=self.llm,
            template=self.PROMPT,
        )

        self.chain.verbose = True

# Automating book writing

## Characters

In [4]:
class MainCharacterChain(BaseStructureChain):
    PROMPT = """
    You are provided with the resume of a person.
    Describe the person's profile in a few sentences and include that person's name.

    Resume: {text}

    Profile:
"""

    def load_resume(self, file_path: str) -> Document:
        loader = PyPDFLoader(file_path)
        return loader.load_and_split()

    def run(self, file_path: str):
        docs = self.load_resume(file_path)
        resume = "\n\n".join([doc.page_content for doc in docs])

        return self.chain.predict(
            text=resume
        )

## The title

In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain.docstore.document import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

class TitleChain(BaseStructureChain):
    PROMPT = """
    Your job is to generate the title for a novel about the following subject and main character.
    Return a title and only a title!
    The title should be consistent with the genre of the novel.
    The title should be consistent with the style of the author.

    Subject: {subject}
    Genre: {genre}
    Author: {author}

    Main character's profile: {profile}

    Title:
"""
    def __init__(self) -> None:
        self.llm = ChatOpenAI()
        self.chain = LLMChain.from_string(
            llm=self.llm,
            template=self.PROMPT,
        )

        self.chain.verbose = True

    def load_resume(self, file_path: str) -> Document:
        loader = PyPDFLoader(file_path)
        return loader.load_and_split()

    def run(self, subject: str, genre: str, author: str, profile: str):
        return self.chain.predict(
            subject=subject,
            genre=genre,
            author=author,
            profile=profile,
        )

## The plot

In [6]:
class PlotChain(BaseStructureChain):
    PROMPT = """
    Your job is to generate the plot for a novel. Return a plot and only a plot.
    Describe the full plot of the story and don't hesitate to create new characters to make it compelling.
    You are provided with the subject, title, and main character's profile.
    Make sure that the main character is at the center of the story.
    The plot should be consistent with the genre of the novel.
    The plot should be consistent with the style of the author.

    Consider the following attributes to write an exciting story:
    {features}

   
    Subject: {subject}
    Genre: {genre}
    Author: {author}

    Title: {title}
    Main character's profile: {profile}

    Plot:
    """

    HELPER_PROMPT = """
    Generate a list of attributes that characterized an exciting story.

    List of attributes:"""

    def run(self, subject: str, genre: str, author: str, profile: str, title: str):
        features = ChatOpenAI().predict(self.HELPER_PROMPT)

        return self.chain.predict(
            subject=subject,
            genre=genre,
            author=author,
            profile=profile,
            title=title,
            features=features,
        )
        
    

## Chapter list

In [8]:
class ChapterListChain(BaseStructureChain):
    PROMPT = """
    Your job is to generate a list of chapters.
    ONLY the list and nothing more!

    You are provided with a title, a plot, and a main character for a novel.
    Gnerate a list of chapters describing the plot of that novel.
    Make sure the chapters are consistent with the genre of the novel.
    Make sure the chapters are consistent with the style of the author.

    Follow this template:

    Prologue: [description of the prologue]
    Chapter 1: [description of the first chapter]
    ...
    Epilogue: [description of the epilogue]

    Make sure the chapter is followed by the character `:` and its description. For example `Chapter 1: [description of the first chapter]`

    Subject: {subject}
    Genre: {genre}
    Author: {author}
    
    Title: {title}
    Main character's profile: {profile}

    Plot: {plot}

    Return the chapter list and only the chapter list:
    """

    def run(self, subject: str, genre: str, author: str, profile: str, title: str, plot: str) -> dict:
        response = self.chain.predict(
            subject=subject,
            genre=genre,
            author=author,
            profile=profile,
            title=title,
            plot=plot,
        )

        return self.parse(response)

    def parse(self, response: str) -> dict:
        chapter_list = response.strip().split("\n")
        chapter_list = [chapter for chapter in chapter_list if ":" in chapter]

        return dict([
            chapter.strip().split(":") for chapter in chapter_list
        ])

## The Chapter's plot

In [16]:
class ChapterPlotChain(BaseEventChain):
    HELPER_PROMPT = """
    Generate a list of attributes that characterized an exciting story.
    List of attributes:"""

    PROMPT = """
    You are a writer and your job is to generate the prot for one and only one chapter of a novel.

    You are provided with a title, a plot, and a main character for a novel.
    Additionally, you are provided with the plots of the previous chapters and the outline of the novel.

    Each chapter should have its own arc, but it should also be consistent with the other chapters, and overall plot of the novel.

    The summary should be consistent with the genre of the novel.
    The summary should be consistent with the style of the author.

    Consider the following attributes to write an exciting story:
    {features}

    Subject: {subject}
    Genre: {genre}
    Author: {author}

    Title: {title}
    Main character's profile: {profile}

    Novel's plot: {plot}

    Outline:
    {outline}

    Chapter plots:
    {summaries}

    Return the plot for the chapter and only the plot:
    Plot for chapter {chapter}:
    """

    def run(self, subject: str, genre: str, author: str, profile: str, title: str, plot: str, summaries_dict: dict, chapter_dict: dict, chapter:str) -> str:
        features = ChatOpenAI().predict(self.HELPER_PROMPT)

        outline = "\n".join([
            f"{chapter}: {description}" for chapter, description in chapter_dict.items()
        ])

        summaries = "\n\n".join([
            f"Plot of {chapter}: {summary}" for chapter, summary in summaries_dict.items()
        ])

        return self.chain.predict(
            subject=subject,
            genre=genre,
            author=author,
            profile=profile,
            title=title,
            plot=plot,
            outline=outline,
            summaries=summaries,
            chapter=chapter,
            features=features,
        )

## Event

In [24]:
class EventChain(BaseEventChain):
    PROMPT = """
    You are a writer and your job is to come up with a detailed list of events happens in the current chapter of a novel.
    Those events describe the plot of the chapter, and the actions of the different characters in chronological order.
    You are provided with the title, the main plot of the novel, the main character, and a summary of that chapter.
    Additionally, you are provided with the list of the events that were outlined in the previous chapters.
    The event list should be consistent with the genre of the novel.
    The event list should be consistent with the style of the author.

    The each element of that list should be returhned on different lines.
    Follow this template:

    Event 1
    Event 2
    ...
    Final event

    Subject: {subject}
    Genre: {genre}
    Author: {author}

    Title: {title}
    Main character's profile: {profile}

    Novel's plot: {plot}

    Events you outlined for previous chapters: {previous_events}

    Summary of the current chapter: {summary}

    Return the events and only the events!
    Event list forr that chapter:

"""
    def run(self, subject: str, genre: str, author: str, profile: str, title: str, plot: str, event_dict: dict, summary: str) -> list[str]:
        previous_events = ""
        for chapter, events in event_dict.items():
            previous_events += "\n" + chapter

            for event in events:
                previous_events += "\n" + event

        response = self.chain.predict(
            subject=subject,
            genre=genre,
            author=author,
            profile=profile,
            title=title,
            plot=plot,
            previous_events=previous_events,
            summary=summary,
        )

        event_list = response.strip().split("\n")

        return [event for event in event_list if event.strip()]
    
    

## Writing the book

In [33]:
class WriteChain(BaseEventChain):
    PROMPT = """
    You are a novel writer. Thie is described by a list of events.
    You have already written the novel up to the last event.
    Your job is to generate the paragraphs of the novel about the new event.
    You are provided with the title, a novel's plot, a description of the main character, and a summary of the current chapter.
    Make sure the paragraphs are consistent with the plot of the chapter.
    Additionally you are provided with the list of the events you have already written about.
    The paragraphs should be consistent with the genre of the novel.
    The paragraphs should be consistent with the style of the author.

    Genre: {genre}
    Author: {author}

    Title: {title}
    Main character's profile: {profile}

    Novel's plot: {plot}
    Previous events: {previous_events}

    Current chapter summary: {summary}

    Previous paragraphs: {previous_paragraphs}

    New event you need to write about: {current_event}

    Paragarphs of the novel describing that event:
    """

    def run(self, genre: str, author: str, title: str, profile: str, plot: str, previous_events: list[str], summary: str, previous_paragraphs: str, current_event: str) -> str:
        return self.chain.predict(
            genre=genre,
            author=author,
            title=title,
            profile=profile,
            plot=plot,
            previous_events="\n".join(previous_events),
            summary=summary,
            previous_paragraphs=previous_paragraphs,
            current_event=current_event,
        )


## Main run

In [31]:
def get_structure(subject: str, genre: str, author: str, profile) -> tuple[str, str, dict]:
    title_chain = TitleChain()
    plot_chain = PlotChain()
    chapters_chain = ChapterListChain()

    title = title_chain.run(subject, genre, author, profile)
    plot = plot_chain.run(subject, genre, author, profile, title)
    chapters = chapters_chain.run(subject, genre, author, profile, title, plot)

    return title, plot, chapters

def get_events(subject: str, genre: str, author: str, profile: str, title: str, plot: str, chapter_dict: dict) -> tuple[dict, dict]:
    chapter_plot_chain = ChapterPlotChain()
    event_chain = EventChain()

    summaries_dict = {}
    event_dict = {}
    for chapter, _ in chapter_dict.items():
        summaries_dict[chapter] = chapter_plot_chain.run(
            subject=subject,
            genre=genre,
            author=author,
            profile=profile,
            title=title,
            plot=plot,
            summaries_dict=summaries_dict,
            chapter_dict=chapter_dict,
            chapter=chapter,
        ) 

        event_dict[chapter] = event_chain.run(
            subject=subject,
            genre=genre,
            author=author,
            profile=profile,
            title=title,
            plot=plot,
            event_dict=event_dict,
            summary=summaries_dict[chapter],
        )

    return summaries_dict, event_dict

def write_book(genre: str, author: str, title: str, profile: str, plot: str, summaries_dict: dict, event_dict: str) -> str:
    writer_chain = WriteChain()
    previous_events = []
    book = {}
    paragraphs = ""

    for chapter, event_list in event_dict.items():
        book[chapter] = []

        for event in event_list:
            paragraphs = writer_chain.run(
                genre=genre,
                author=author,
                title=title,
                profile=profile,
                plot=plot,
                previous_events=previous_events,
                summary=summaries_dict[chapter],
                previous_paragraphs=paragraphs,
                current_event=event,
            )

            previous_events.append(event)
            book[chapter].append(paragraphs)

    return book

In [ ]:
main_character_chain = MainCharacterChain()

subject = "Machine War"
author = "Ernest Hemingway"
genre = "Science Fiction"

profile = main_character_chain.run("./docs/Profile.pdf")

title, plot, chapter_dict = get_structure(subject, genre, author, profile)
summaries_dict, event_dict = get_events(subject, genre, author, profile, title, plot, chapter_dict)


In [ ]:
book = write_book(genre, author, title, profile, plot, summaries_dict, event_dict)

## Write to file

In [35]:
import docx

class DocWriter:
    def __init__(self) -> None:
        self.doc = docx.Document()

    def write_doc(self, book, chapter_dict, title):
        self.doc.add_heading(title, 0)

        for chapter, paragraph_list in book.items():
            description = chapter_dict[chapter]

            chapter_name = f"Chapter {chapter.strip()}: {description.strip()}"

            self.doc.add_heading(chapter_name, 1)

            text = "\n\n".join(paragraph_list)
            self.doc.add_paragraph(text)

        self.doc.save('./docs/book.docx')

In [36]:
doc_writer = DocWriter()

doc_writer.write_doc(book, chapter_dict, title)